<a href="https://colab.research.google.com/github/Nithyarajoman/Machine-learning-Trial/blob/main/Effective_data_communicated_no_hierarchy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models

In [2]:
# Read data
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float')/255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float')/255

4422102/4422102 [==============================] - 0s 0us/step


In [3]:
# Create Model
model_hierarchy = models.Sequential(
    [        
        layers.Conv2D(32, kernel_size=(3, 3),
                      input_shape=(28, 28, 1),
                      strides=(2,2), padding="same",
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation(activation="relu"),
        layers.Conv2D(64, kernel_size=(3, 3),
                      strides=(2,2), padding="same",
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation(activation="relu"),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ]
)

In [4]:
# Train
model_hierarchy.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model_hierarchy.fit(x=x_train, y=y_train, epochs=5, batch_size=128, validation_split=0.1 )
score = model_hierarchy.evaluate(x_test, y_test, verbose=0)

Epoch 1/5
422/422 [==============================] - 26s 59ms/step - loss: 0.5369 - accuracy: 0.8127 - val_loss: 0.6312 - val_accuracy: 0.7780
Epoch 2/5
422/422 [==============================] - 24s 56ms/step - loss: 0.3646 - accuracy: 0.8708 - val_loss: 0.3097 - val_accuracy: 0.8868
Epoch 3/5
422/422 [==============================] - 25s 59ms/step - loss: 0.3228 - accuracy: 0.8833 - val_loss: 0.2945 - val_accuracy: 0.8947
Epoch 4/5
422/422 [==============================] - 23s 56ms/step - loss: 0.2996 - accuracy: 0.8921 - val_loss: 0.2712 - val_accuracy: 0.9018
Epoch 5/5
422/422 [==============================] - 24s 56ms/step - loss: 0.2828 - accuracy: 0.8961 - val_loss: 0.2624 - val_accuracy: 0.9082


In [5]:
model_hierarchy.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 32)        288       
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 14, 14, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18432     
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 64)         256       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 7, 7, 64)          0

In [6]:
print(model_hierarchy.inputs)

[<KerasTensor: shape=(None, 28, 28, 1) dtype=float32 (created by layer 'conv2d_input')>]


In [7]:
# Creating model
# Building new model with multiple output layers
all_layers = [layer.output for layer in [model_hierarchy.get_layer(index=0),
                                        model_hierarchy.get_layer(index=1),
                                        model_hierarchy.get_layer(index=2),
                                        model_hierarchy.get_layer(index=3),
                                        model_hierarchy.get_layer(index=4),
                                        model_hierarchy.get_layer(index=5),
                                        model_hierarchy.get_layer(index=6),
                                        model_hierarchy.get_layer(index=7),
                                        model_hierarchy.get_layer(index=8)]]
all_model = tf.keras.Model(inputs=model_hierarchy.inputs, outputs=all_layers)
all_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_input (InputLayer)   [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 32)        288       
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 14, 14, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18432     
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 64)         256       
 hNormalization)                                             

In [13]:
aux = all_model(x_train[:100])
print(aux[5].shape)
len(aux)

(100, 7, 7, 64)


9

In [15]:
%pylab
plt.imshow(aux[5][1,:,:,1]) 

Using matplotlib backend: agg
Populating the interactive namespace from numpy and matplotlib


In [16]:
x= aux[5]

In [17]:
y=tf.math.count_nonzero(x)
total=tf.size(x)
print(total.numpy())
print(y.numpy())
zeroes = total.numpy() - y.numpy() 
print("No of elements that are zeroes in 5th layer: %i " %zeroes)

313600
134145
No of elements that are zeroes in 5th layer: 179455 


In [8]:
# Function to calculate the number of non zer elements in each layer

def non_zeroes_layer(l):
    
    prob_non_zeroes=tf.math.count_nonzero(l)
    total=tf.size(l)
    # prob_non_zeroes = y.numpy()/total.numpy()
    # print(prob_non_zeroes.numpy()) 
    return prob_non_zeroes.numpy()
    # zeroes = total.numpy() - y.numpy()
    # print("No of elements that are zeroes in 5th layer: %i " %zeroes)

In [9]:
# Passing layers to the function to get the count

def summary_communication(model):
  aux = model(x_train[:100])
  print(aux)
  print('%25s | %6s' % (
      'Layer Name', 'Effective communication'))
  print('*' * 100)

  for l,value in enumerate(aux):
    # if l != 0:
      # if ('Sequential' in str(l)):
      #   for ib_layer in l.layers:
      #     x = aux[ib_layer]
      #     probability = non_zeroes_layer(x)
      #     print('%25s(%5s) | %5.4f' % (
      #             l.name,ib_layer.name, probability ))
          
      # else:
        x = aux[l]
        probability = non_zeroes_layer(x)
        print(probability)
    
  return

In [11]:
summary_communication(all_model)

[<tf.Tensor: shape=(100, 14, 14, 32), dtype=float32, numpy=
array([[[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  